# Use the finetuned LLM in a New Session

In [1]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

In [2]:
import torch
from spam_classifier_utils import GPTModel

# Instantiate model (random weights, not trained)
model = GPTModel(BASE_CONFIG)
num_classes = 2
model.out_head = torch.nn.Linear(in_features=768, out_features=num_classes)


##########################################
# Exercise: Load finetuned model weights
##########################################

# Load saved file
model_state_dict = torch.load("review_classifier.pth")
# Apply the state dictionary to your model instance
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [3]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [4]:
import tiktoken
from spam_classifier_utils import classify_review

tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
text_1 = (
    "Congratulations! You have WON a $500 Amazon gift card"
    " in our exclusive lucky draw!"
)

print(classify_review(text_1, model, tokenizer, device, pad_length=120))

Positive


In [6]:
text_2 = (
    "Reminder: School Parent-Teacher meeting tonight at 7 PM in the main "
    "auditorium. We look forward to seeing you there and discussing your "
    "child's progress."
)

print(classify_review(text_2, model, tokenizer, device, pad_length=120))

Negative
